# Near-Inertial Activity Forecast

## Quick links

- [Near-inertial current maps](#Max-near-inertial-speed-over-good-and-whole-forecast-period)
- [Near-inertial current time series](#Near-inertial-current-timeseries-for-buoy-locations)
- [Atmospheric conditions](#Atmospheric-conditions-over-forecast-period)

## Details

To see how we got here, check the notebooks with the [loading the GFS atmospheric forecast data](./010_download_GFS_data.html), 
the [loading of the buoy data](./011_download_buoy_data.html), and the [evaluation of the slab-ocean model](./020_run_slab_model.html).

Details: <https://github.com/willirath/nia-prediction-low-latitudes/>

## Parameters

The following parameters determine the regional coverage of the forecast and the positions for which we plot detailed time series.
They also determine the location of (temporary) data files, and details for the parallelization with [Dask](https://dask.org/).

In [ ]:
# parameters

# regional coverage
lat_min, lat_max = -35, 35
lon_min, lon_max = 0, 360

# good forecasts reach
good_forecast_days = 7  # 7 days

# additional buoy positions to be plotted
# The ones found in the downloaded data will be shown anyway.
added_buoy_positions = [
    {"lat": 20.0, "lon": -38.0},
    {"lat": 15.0, "lon": -38.0},
    {"lat": 21.0, "lon": -23.0},
    {"lat": 12.0, "lon": -23.0},
    {"lat": -6.0, "lon": -10.0},
    {"lat": -10.0 ,"lon": -10.0},
]

# data files
GFS_zarr_store = "tmp_GFS.zarr"
slab_zarr_store = "tmp_slab.zarr"
buoy_file_name = "tmp_buoy_data"
buoy_positions_file = "tmp_buoy_positions.csv"


# dask specifics
dask_kwargs = {"n_workers": 1, "threads_per_worker": 2, "memory_limit": 6e9}

## Technial Preamble

Before doing any calculations, we'll need to import a few modules. We'll also start a Dask cluster for parallel execution.

In [ ]:
# dask
from dask.distributed import Client

# plotting
from bokeh.models.formatters import DatetimeTickFormatter
import cartopy.crs as ccrs
import cmocean
import geoviews as gv
import holoviews as hv
import hvplot.xarray, hvplot.pandas

# numerics
import numpy as np
import pandas as pd
import xarray as xr

# aux
from functools import reduce
from operator import add

In [ ]:
# create Dask cluster
client = Client(**dask_kwargs)
client

## Get buoy locations from the buoy data set

In [ ]:
buoy_positions = pd.read_csv(buoy_positions_file)
added_buoy_positions =pd.DataFrame.from_records(added_buoy_positions)
added_buoy_positions["lon"] = (360.0 + added_buoy_positions["lon"]) % 360.0
buoy_positions = buoy_positions.merge(added_buoy_positions, how="outer")
buoy_positions.head(3)

## Load the GFS and slab-model data

In [ ]:
ds_GFS = xr.open_zarr(GFS_zarr_store)
ds_slab = xr.open_zarr(slab_zarr_store)

### Find start of forecast period

We'll need the time stamp of the start of the forecasting data.

In [ ]:
start_of_forecast = (~ds_GFS["is_forecast"].astype(bool)).sum().compute().data
start_of_forecast = ds_GFS["time"].data[max(0, start_of_forecast-1)]
print(start_of_forecast)

In [ ]:
good_forecast_time = np.timedelta64(good_forecast_days, "D")

## Restrict regionally

In [ ]:
ds_GFS = ds_GFS.sel(
    lat=slice(lat_max, lat_min),
    lon=slice(lon_min, lon_max),
)
ds_slab = ds_slab.sel(
    lat=slice(lat_max, lat_min),
    lon=slice(lon_min, lon_max),
)

In [ ]:
ds_slab

In [ ]:
buoy_positions = buoy_positions.where(
    buoy_positions["lat"].apply(pd.Interval(lat_min, lat_max).__contains__)
    & buoy_positions["lon"].apply(pd.Interval(lon_min, lon_max).__contains__)
).dropna()

## Max near-inertial speed over good and whole forecast period

We'll plot the time-maximum of near-inertial speed for the good forecast period and for the whole forecast period.

In [ ]:
slab_umag_good_forecast_max = ds_slab["umag_slab"].sel(
    time=slice(start_of_forecast, start_of_forecast + good_forecast_time)
).max("time")
slab_umag_whole_forecast_max = ds_slab["umag_slab"].sel(
    time=slice(start_of_forecast, None)
).max("time")

near_inertial_max_plots = (
    (
        slab_umag_good_forecast_max.hvplot(
            x="lon", y="lat", z="umag_slab",
            clim=(0, 1.5),
            cmap=cmocean.cm.speed,
            frame_width=800,
            hover=False,
            geo=True, coastline=True,
            crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(),
            title="Near-inertial speed max [m/s], good forecast"
        )
        + slab_umag_whole_forecast_max.hvplot(
            x="lon", y="lat", z="umag_slab",
            clim=(0, 1.5),
            cmap=cmocean.cm.speed,
            frame_width=800,
            hover=False,
            geo=True, coastline=True,
            crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(),
            title="Near-inertial speed max [m/s], whole forecast"
        )
    ) * buoy_positions.hvplot.points(
        y="lat", x="lon", geo=True, coastline=True,
        marker='circle',
        fill_color=None, line_color="black",
        line_width=1, size=60,
    ) * gv.feature.grid()
).cols(1)

In [ ]:
display(near_inertial_max_plots)

## Near-inertial current timeseries for buoy locations

In [ ]:
time_series_plots = []
time_formatter = DatetimeTickFormatter(
    months='%b %Y', days='%b %d'
)

forecast_spans = (
    hv.VSpan(
        start_of_forecast, start_of_forecast + good_forecast_time
    ).opts(padding=0, color='lightgray')
    * hv.VSpan(
        start_of_forecast + good_forecast_time, None
    ).opts(padding=0, color='pink')
)

for lat, lon in zip(buoy_positions["lat"], buoy_positions["lon"]):
    name = f"{lat}N {lon}E"
    buoy_ds = ds_slab.sel(lat=lat, lon=lon, method="nearest")
    buoy_ds["U20"] = ds_GFS["U20"].sel(lat=lat, lon=lon, method="nearest")
    buoy_ds["V20"] = ds_GFS["V20"].sel(lat=lat, lon=lon, method="nearest")
    
    if (buoy_ds["umag_slab"].max("time").isnull().data.compute()):
        continue
    time_series_plots.append(
        (
            (
                forecast_spans.redim.label(y="u_slab")
                * buoy_ds["u_slab"].hvplot.line(label="zonal near-inertial current")
                * buoy_ds["v_slab"].hvplot.line(label="meridional near-inertial current")
                * buoy_ds["umag_slab"].hvplot.line(label="near-inertial speed")
            ).options(
                width=800, height=160, show_grid=True,
                xaxis=None,
                legend_cols=False, legend_position='right',
                ylabel="current [m/s]", title=name
            )
            + (
                forecast_spans.redim.label(y="U20")
                * buoy_ds["U20"].hvplot.line(label="zonal wind (20m)")
                * buoy_ds["V20"].hvplot.line(label="meridional wind (20m)")
            ).options(
                width=800, height=160, show_grid=True,
                xformatter=time_formatter,
                legend_cols=False, legend_position='right',
                ylabel="wind [m/s]", xlabel=""
            )
        )
    )

time_series_plots = reduce(add, time_series_plots).cols(1)

In [ ]:
display(time_series_plots)

## Atmospheric conditions over forecast period

To get a feeling for the atmospheric conditions, we'll plot sea-level pressure anomalies every 12 hours for 3 days before and throughout the whole forecast period.

Anomalies are calculated relative to the whole data period (usually 30+14 days).

In [ ]:
SLP_mean = ds_GFS["SLP"].mean("time")
SLP_anomaly = (ds_GFS["SLP"] - SLP_mean)

In [ ]:
plot_every = np.timedelta64(12, "h")
max_iter = ((SLP_anomaly.coords["time"].max("time") - start_of_forecast) / plot_every).item() // 1 + 1

plot_times = [
    (start_of_forecast + n * plot_every)
    for n in range(-6, int(max_iter))
]

plots = []

for plot_time in plot_times:
    title = f"SLP anomaly [hPa], {pd.Timestamp(plot_time).strftime('%Y-%m-%d %H:%M:%S UTC')}"
    if plot_time > start_of_forecast:
        title += f"\t(forecast + {(plot_time - start_of_forecast) / np.timedelta64(1, 'h')}h)"
    try:
        plots.append(
            (
                SLP_anomaly.sel(time=plot_time, method="nearest").compute().hvplot(
                    clim=(-10, 10),
                    cmap=cmocean.cm.delta,
                    frame_width=800,
                    geo=True, coastline=True,
                    crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(),
                    hover=False
                )
                * buoy_positions.hvplot.points(
                    y="lat", x="lon", geo=True, coastline=True,
                    marker='circle',
                    fill_color=None, line_color="black",
                    line_width=1.5, size=60,
                )
                * gv.feature.grid()
            ).opts(
                title=title,
                show_grid=True
            )
        )    
    except Exception as e:
        print(f"for {plot_time} I got: {e}")
    
slp_plot = reduce(add, plots).cols(1)

In [ ]:
display(slp_plot)

---

In [ ]:
!echo "Finished: $(date -Ins) (UTC)"

---
See https://github.com/willirath/nia-prediction-low-latitudes for details.